In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
img_array = cv2.imread('T\Close_Eye\s0001_00456_0_0_0_0_0_01.png')

In [ ]:
datadirectory = "Test/"
classes = ["Close_Eye", "Open_Eye"]
for category in classes:
    path = os.path.join(datadirectory, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
        plt.imshow(img_array, cmap = "gray")
        plt.show()
        break
    break

In [ ]:
img_size = 224

new_array = cv2.resize(backtorgb, (img_size, img_size))
plt.imshow(new_array, cmap = "gray")
plt.show()

# Reading all the images and converting them into an array for data and labels

In [ ]:
training_data = []
def create_training_data():
    for category in classes:
        path = os.path.join(datadirectory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
                new_array = cv2.resize(backtorgb, (img_size, img_size))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

In [ ]:
create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
import random

random.shuffle(training_data)

In [ ]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 3)

In [ ]:
X.shape

In [ ]:
# normalise the data
X = X/225.0;

In [ ]:
y = np.array(y)

In [ ]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

# Deep learning model for training - Training Learning

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = tf.keras.applications.mobilenet.MobileNet()

In [ ]:
model.summary()

# Transfer Learning

In [ ]:
base_input = model.layers[0].input

In [ ]:
base_output = model.layers[-4].output

In [ ]:
flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(flat_layer)  #one node (1 or 0)
final_output = layers.Activation('sigmoid')(final_output)

In [ ]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

In [ ]:
new_model.summary()

# Settings for binary classification (open/close)

In [ ]:
new_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
new_model.fit(X, y, epochs = 1)

In [ ]:
new_model.save('my_model.hS')

In [ ]:
new_model = tf.keras.models.load_model('my_model.hS')

# Checking the network for prediction

In [ ]:
img_array = cv2.imread('Test\Close_Eye\s0001_00474_0_0_0_0_0_01.png', cv2.IMREAD_GRAYSCALE)
backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
new_array = cv2.resize(backtorgb, (img_size, img_size))

In [ ]:
X_input = np.array(new_array).reshape(1, img_size, img_size, 3)

In [ ]:
X_input.shape

In [ ]:
plt.imshow(new_array)

In [ ]:
X_input = X_input/255.0

In [ ]:
prediction = new_model.predict(X_input)

In [ ]:
prediction

# Checking for unknown images

In [ ]:
img = cv2.imread('Test.png')

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
eyes = eye_cascade.detectMultiScale(gray,1.1,2)

In [ ]:
for (x, y, w, h) in eyes:
    cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Croping the eye image

In [ ]:
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
eyes = eye_cascade.detectMultiScale(gray,1.1,4)
for x, y, w, h in eyes:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyess = eye_cascade.detectMultiScale(roi_gray)
    if len(eyess) == 0:
        print("Eyes not detected")
    else:
        for (ex,ey,ew,eh) in eyess:
            eyes_roi = roi_color[ey:ey+eh, ex:ex+ew]

In [ ]:
plt.imshow(cv2.cvtColor(eyes_roi, cv2.COLOR_BGR2RGB))

In [ ]:
eyes_roi.shape

In [ ]:
final_img = cv2.resize(eyes_roi, (224,224))
final_img = np.expand_dims(final_img, axis = 0)
final_img = final_img/255.0

In [ ]:
final_img.shape

In [ ]:
new_model.predict(final_img)

# On live video

In [4]:
import cv2
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
path = 'haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
new_model = tf.keras.models.load_model('my_model.hS')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray,1.1,4)
    for ax, ay, aw, ah in eyes:
        cv2.rectangle(frame, (ax,ay),(ax+aw, ay+ah), (0,255,0),2)
        roi_gray = gray[ay:ay+ah, ax:ax+aw]
        roi_color = frame[ay:ay+ah, ax:ax+aw]
        eyess = eye_cascade.detectMultiScale(roi_gray)
        if len(eyess) == 0:
            print("Eyes not detected")
        else:
            for (ex,ey,ew,eh) in eyess:
                eyes_roi = roi_color[ey:ey+eh, ex:ex+ew]
    final_img = cv2.resize(eyes_roi, (224,224))
    final_img = np.expand_dims(final_img, axis = 0)
    final_img = final_img/255.0

    predictions = new_model.predict(final_img)
    if(predictions>0.8):
        status = "Open Eyes"
        #print("Open")
    else:
        status = "Closed Eyes"
        #print("Closed")

    print(faceCascade.empty())
    faces = faceCascade.detectMultiScale(gray,1.1,4)

    # Draw a rectangle around the faces
    for(x, y, w, h) in faces:
        cv2.rectangle(frame, (x,y),(x+w, y+h), (0,255,0),2)
    font = cv2.FONT_HERSHEY_COMPLEX

    cv2.putText(frame, status, (10,40), font, 1, (0,0,255), 3, cv2.LINE_4)
    cv2.imshow('Drowsiness Detection', frame)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Eyes not detected
1/1 [==============================] - 1s 589ms/step
Open
False
1/1 [==============================] - 0s 57ms/step
Open
False
Eyes not detected
1/1 [==============================] - 0s 64ms/step
Open
False
1/1 [==============================] - 0s 35ms/step
Open
False
1/1 [==============================] - 0s 33ms/step
Open
False
Eyes not detected
1/1 [==============================] - 0s 40ms/step
Open
False
1/1 [==============================] - 0s 33ms/step
Open
False
1/1 [==============================] - 0s 42ms/step
Open
False
1/1 [==============================] - 0s 40ms/step
Open
False
1/1 [==============================] - 0s 41ms/step
Open
False
1/1 [==============================] - 0s 49ms/step
Open
False
1/1 [==============================] - 0s 43ms/step
Open
False
1/1 [==============================] - 0s 48ms/step
Open
False
Eyes not detected
1/1 [==============================] - 0s 38ms/step
Open
False
1/1 [==============================] - 0s 44m

# If eyes are closed for unusual time, alarm generated

In [13]:
import cv2
import tensorflow as tf
import cv2
import numpy as np
import winsound

frequency = 2500
duration = 1000
counter = 0

path = 'haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
new_model = tf.keras.models.load_model('my_model.hS')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray,1.1,4)
    for ax, ay, aw, ah in eyes:
        cv2.rectangle(frame, (ax,ay),(ax+aw, ay+ah), (0,255,0),2)
        roi_gray = gray[ay:ay+ah, ax:ax+aw]
        roi_color = frame[ay:ay+ah, ax:ax+aw]
        eyess = eye_cascade.detectMultiScale(roi_gray)
        if len(eyess) == 0:
            print("Eyes not detected")
        else:
            for (ex,ey,ew,eh) in eyess:
                eyes_roi = roi_color[ey:ey+eh, ex:ex+ew]
    final_img = cv2.resize(eyes_roi, (224,224))
    final_img = np.expand_dims(final_img, axis = 0)
    final_img = final_img/255.0

    predictions = new_model.predict(final_img)
    if(predictions>0.5):
        status = "Open Eyes"
        #print("Open")
        cv2.putText(frame, status, (10,40), font, 1, (0,0,255), 3, cv2.LINE_4)
        cv2.putText(frame, 'Active', (10,80), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,0), 2)
    else:
        counter = counter + 1
        status = "Closed Eyes"
        #print("Closed")
        cv2.putText(frame, status, (10,40), font, 1, (0,0,255), 3, cv2.LINE_4)
        if counter > 30:
            cv2.putText(frame, 'Sleep Alert!!!', (10,80), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,0), 2)
            winsound.Beep(frequency, duration)
            counter = 0

    print(faceCascade.empty())
    faces = faceCascade.detectMultiScale(gray,1.1,4)

    # Draw a rectangle around the faces
    for(x, y, w, h) in faces:
        cv2.rectangle(frame, (x,y),(x+w, y+h), (0,255,0),2)
    font = cv2.FONT_HERSHEY_COMPLEX

    cv2.putText(frame, status, (10,40), font, 1, (0,0,255), 3, cv2.LINE_4)
    cv2.imshow('Drowsiness Detection', frame)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Eyes not detected
Eyes not detected
1/1 [==============================] - 0s 441ms/step
False
Eyes not detected
Eyes not detected
1/1 [==============================] - 0s 71ms/step
False
Eyes not detected
1/1 [==============================] - 0s 68ms/step
False
Eyes not detected
1/1 [==============================] - 0s 63ms/step
False
1/1 [==============================] - 0s 62ms/step
False
1/1 [==============================] - 0s 60ms/step
False
1/1 [==============================] - 0s 63ms/step
False
1/1 [==============================] - 0s 57ms/step
False
1/1 [==============================] - 0s 75ms/step
False
1/1 [==============================] - 0s 71ms/step
False
1/1 [==============================] - 0s 58ms/step
False
1/1 [==============================] - 0s 73ms/step
False
1/1 [==============================] - 0s 58ms/step
False
Eyes not detected
1/1 [==============================] - 0s 60ms/step
False
Eyes not detected
1/1 [==============================] - 0s 6

In [ ]:
if (Predictions>0):

status "Open Eyes"

cv2.putText(frame,

status,

(150, 150),

font, 3,

(0, 255, 0),

2,

cv2.LINE 4)

x1,y1,wl,h1 0,0,175,75

# Draw black background rectangle

cv2.rectangle(frame, (x1, x1), (x1+ w1, y1+ h), (0,0,0), -1)

# Add text

cv2.putText(frame, 'Active', (x1+ int(w1/10),y1+ int (h1/2)), cv2.FONT HERSHEY SIMPLEX, 8.7, (0,255,0), 2)

else:

counter counter + 1

status = "Closed Eyes"

cv2.putText(frame,

status,

(150, 150),

font, 3, (0, 0, 255),

2,

cv2.LINE_4)

cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

if counter>5:

x1,y1,wi,h1 0,0,175,75

#Draw black background rectangle

cv2.rectangle(frame, (x1, x1), (x1+ w1, y1+ h1), (0,0,0), -1)

# Add text

cv2.putText(frame, 'Sleep Alert !!, (x1+ int(w1/10),y1+ int (h1/2)), cv2.FONT HERSHEY SIMPLEX, 8.7, (0,8,255), 2) winsound.Beep(frequency, duration)

counter = 0